# Ultramarathon Data Analysis

## Data Cleanup and Preperation

This dataset can be downloaded from kaggle here: https://www.kaggle.com/datasets/aiaiaidavid/the-big-dataset-of-ultra-marathon-running/

In [1]:
import numpy as np
import pandas as pd
import re

races = pd.read_csv('TWO_CENTURIES_OF_UM_RACES.csv', low_memory=False)
races.head()

,Year of event,Event dates,Event name,Event distance/length,Event number of finishers,Athlete performance,Athlete club,Athlete country,Athlete year of birth,Athlete gender,Athlete age category,Athlete average speed,Athlete ID
0,2018,06.01.2018,Selva Costera (CHI),50km,22,4:51:39 h,Tnfrc,CHI,1978.0,M,M35,10.286,0
1,2018,06.01.2018,Selva Costera (CHI),50km,22,5:15:45 h,Roberto Echeverría,CHI,1981.0,M,M35,9.501,1
2,2018,06.01.2018,Selva Costera (CHI),50km,22,5:16:44 h,Puro Trail Osorno,CHI,1987.0,M,M23,9.472,2
3,2018,06.01.2018,Selva Costera (CHI),50km,22,5:34:13 h,Columbia,ARG,1976.0,M,M40,8.976,3
4,2018,06.01.2018,Selva Costera (CHI),50km,22,5:54:14 h,Baguales Trail,CHI,1992.0,M,M23,8.469,4


### 1. Rename columns and convert data types

In [2]:
races = races.rename(mapper={
    'Year of event': 'event_year',
    'Event dates': 'event_dates',
    'Event name': 'event_name',
    'Event distance/length': 'event_distance_or_length',
    'Event number of finishers': 'event_num_finishers',
    'Athlete performance': 'athlete_performance',
    'Athlete club': 'athlete_club',
    'Athlete country': 'athlete_country',
    'Athlete year of birth': 'athlete_birth_year',
    'Athlete gender': 'athlete_gender',
    'Athlete age category': 'athlete_age_category',
    'Athlete average speed': 'athlete_avg_speed',
    'Athlete ID': 'athlete_id'
}, axis=1)
races['athlete_birth_year'] = races['athlete_birth_year'].astype('Int64', errors='ignore')
races.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7461195 entries, 0 to 7461194
Data columns (total 13 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   event_year                int64 
 1   event_dates               object
 2   event_name                object
 3   event_distance_or_length  object
 4   event_num_finishers       int64 
 5   athlete_performance       object
 6   athlete_club              object
 7   athlete_country           object
 8   athlete_birth_year        Int64 
 9   athlete_gender            object
 10  athlete_age_category      object
 11  athlete_avg_speed         object
 12  athlete_id                int64 
dtypes: Int64(1), int64(3), object(9)
memory usage: 747.1+ MB


### 2. Itentify stage races and clean up distance

In [3]:
races['stage_race'] = races['event_distance_or_length'].str.contains('Etappen')
races['stage_days'] = races['event_distance_or_length'].str.extract(r'(\d+)Etappen').astype('Int64')
stage_pattern = re.compile(r'/\d+Etappen')
races['event_distance_or_length'] = races['event_distance_or_length'].str.replace(pat=stage_pattern,repl='',regex=True)
races['event_distance_or_length'] = races['event_distance_or_length'].str.replace(pat=r'([mM]iles?)',repl='mi',regex=True)
races['event_distance_or_length'] = races['event_distance_or_length'].str.replace(pat=r'[kK][mM]?',repl='km',regex=True)

In [4]:
races['distance_based'] = races['event_distance_or_length'].str.contains(r'(?:mi|km)')
races[races['distance_based'] == True]['event_distance_or_length'].unique()

array(['50km', '63.9km', '50mi', ..., '2925.7mi', '355km', '2906.1mi'],
      shape=(1593,), dtype=object)

### 3. Extract Start and End Dates for multi-day events

In [5]:
def extract_dates(datestring):
    # datestring = row['event_dates']
    matching = re.findall(r'(\d+)',datestring)
    end_date = matching[-3] + '/' + matching[-2] + '/' + matching[-1]
    start_date = matching[0] + '/'
    if len(matching) >= 5:
        start_date += matching[1] + '/'
    else:
        start_date += matching[-2] + '/'
    if len(matching) == 6:
        start_date += matching[2]
    else:
        start_date += matching[-1]
    return start_date+'-'+end_date

event_dates = races['event_dates'].apply(extract_dates)
races[['start_date','end_date']] = event_dates.str.split('-',expand=True)

### 4. Clean up event name and extract country code

In [6]:
races['event_country'] = races['event_name'].str.extract(r'\((\w{3})\)')
races['event_name'] = races['event_name'].str.extract(r'(^.+)\([A-Z]{3}\)$')

In [7]:
races.count()

event_year                  7461195
event_dates                 7461195
event_name                  7459569
event_distance_or_length    7460142
event_num_finishers         7461195
athlete_performance         7461193
athlete_club                4634671
athlete_country             7461192
athlete_birth_year          6873034
athlete_gender              7461188
athlete_age_category        6876257
athlete_avg_speed           7460971
athlete_id                  7461195
stage_race                  7460142
stage_days                    91944
distance_based              7460142
start_date                  7461195
end_date                    7461195
event_country               7459569
dtype: int64

### 5. Pull out US races

In [8]:
us_race_results = races[(races['event_country'] == 'USA') & (races['event_year']>=1975)]
us_races = us_race_results[[
    'event_year',
    'event_dates',
    'event_name',
    'event_distance_or_length',
    'event_num_finishers',
    'stage_race',
    'stage_days',
    'start_date',
    'end_date',
    'event_country',
    'distance_based'
]].drop_duplicates().reset_index().drop('index',axis=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27277 entries, 0 to 27276
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   event_year                27277 non-null  int64 
 1   event_dates               27277 non-null  object
 2   event_name                27277 non-null  object
 3   event_distance_or_length  27277 non-null  object
 4   event_num_finishers       27277 non-null  int64 
 5   stage_race                27277 non-null  object
 6   stage_days                123 non-null    Int64 
 7   start_date                27277 non-null  object
 8   end_date                  27277 non-null  object
 9   event_country             27277 non-null  object
 10  distance_based            27277 non-null  object
dtypes: Int64(1), int64(2), object(8)
memory usage: 2.3+ MB


## Popularity of Ultramarathons in the US Since 1975